In [0]:
%run /Workspace/carsdataflow_databricks_notebooks/setup

In [0]:
bronze_path = "abfss://bronze@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/"
silver_path = "abfss://silver@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/"

In [0]:
%sql
-- In SQL you can access the file using the path in backtics as below
select * from parquet.`abfss://bronze@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/`

### Reading Data into Dataframe

In [0]:
# storing the data into dataframe
# you can also use spark.read.format("csv").option("header", "true").load(bronze_path)
df = spark.sql('select * from parquet.`abfss://bronze@adlsforcarsdataflow.dfs.core.windows.net/Carsdata/`')
df.display()

### Applying Transformation

In [0]:
from pyspark.sql.functions import *

In [0]:
# Spliting the Model Id into Model_Category
df = df.withColumn("Model_Category", split(col("Model_ID"), '-')[0])
df.display()


In [0]:
df = df.withColumn("Units_Sold_Per_Unit", col("Revenue")/col("Units_Sold"))
df.display()

### Ad-HOC

In [0]:
# Applying Gropu By & aggregating the data

df.groupBy("Model_Category", "Year").agg(avg("Units_Sold_Per_Unit")).display()

Databricks visualization. Run in Databricks to view.

### Writing Data to the silver layer

In [0]:
df.write.format("Delta").mode("overwrite").save(silver_path)

In [0]:
# checking whether the data is stored or not

delta_df = spark.read.format("delta").load(silver_path)
delta_df.display()